In [1]:
from kiteconnect import KiteConnect
from selenium import webdriver
import time
import os
import re

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
from matplotlib import pyplot as plt
import os
import sqlite3

In [3]:
apikey=""
apisecret=""

In [4]:
kite=KiteConnect(api_key=apikey)

In [5]:
print(kite.login_url())

https://kite.zerodha.com/connect/login?api_key=kdwok1gk1epq3k3k&v=3


In [6]:
data=kite.generate_session("hTyiPdpkXSxQlgP7iBgyQPV6XnCDS2WK",api_secret=apisecret)

In [7]:
data

{'user_type': 'individual',
 'email': 'the.ravina@gmail.com',
 'user_name': 'Ravina Vinayak More',
 'user_shortname': 'Ravina',
 'broker': 'ZERODHA',
 'exchanges': ['CDS', 'BSE', 'NSE', 'NFO', 'BFO', 'MF'],
 'products': ['CNC', 'NRML', 'MIS', 'BO', 'CO'],
 'order_types': ['MARKET', 'LIMIT', 'SL', 'SL-M'],
 'avatar_url': None,
 'user_id': 'LF9990',
 'api_key': 'kdwok1gk1epq3k3k',
 'access_token': '3MmnWLFC3FfWn6UZbl677wwK0Ko6XH9f',
 'public_token': 'NM479rue5cRwIag6HSXkfNofoaxUQJ0E',
 'refresh_token': '',
 'enctoken': '5L6zY67T6J5tP91PQAPEtWBuBwu9hUhLaL/FxYfuqzVsvtDGF2DMTtFzYbFbQTUB5+F3zTyC+acbYOQKqGmEXONHsMTxhrAeyPc0Xff+4nPtzrGPFlnQ/30QigoYRRw=',
 'login_time': datetime.datetime(2023, 5, 14, 15, 40, 39),
 'meta': {'demat_consent': 'physical'}}

In [8]:
kite.set_access_token(data["access_token"])

In [44]:
kite.instruments("NSE")

[{'instrument_token': 264713,
  'exchange_token': '1034',
  'tradingsymbol': 'HANGSENG BEES-NAV',
  'name': 'HANGSENG BEES-NAV',
  'last_price': 0.0,
  'expiry': '',
  'strike': 0.0,
  'tick_size': 0.0,
  'lot_size': 0,
  'instrument_type': 'EQ',
  'segment': 'INDICES',
  'exchange': 'NSE'},
 {'instrument_token': 264969,
  'exchange_token': '1035',
  'tradingsymbol': 'INDIA VIX',
  'name': 'INDIA VIX',
  'last_price': 0.0,
  'expiry': '',
  'strike': 0.0,
  'tick_size': 0.0,
  'lot_size': 0,
  'instrument_type': 'EQ',
  'segment': 'INDICES',
  'exchange': 'NSE'},
 {'instrument_token': 260617,
  'exchange_token': '1018',
  'tradingsymbol': 'NIFTY 100',
  'name': 'NIFTY 100',
  'last_price': 0.0,
  'expiry': '',
  'strike': 0.0,
  'tick_size': 0.0,
  'lot_size': 0,
  'instrument_type': 'EQ',
  'segment': 'INDICES',
  'exchange': 'NSE'},
 {'instrument_token': 264457,
  'exchange_token': '1033',
  'tradingsymbol': 'NIFTY 200',
  'name': 'NIFTY 200',
  'last_price': 0.0,
  'expiry': '',
  '

In [10]:
inst_df=pd.DataFrame(kite.instruments("NSE"))

In [11]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1

In [12]:
def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(inst_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data

In [13]:
def fetchOHLCExtended(ticker,inception_date, interval):
    """extracts historical data and outputs in the form of dataframe
       inception date string format - dd-mm-yyyy"""
    instrument = instrumentLookup(inst_df,ticker)
    from_date = dt.datetime.strptime(inception_date, '%d-%m-%Y')
    to_date = dt.date.today()
    data = pd.DataFrame(columns=['date', 'open', 'high', 'low', 'close', 'volume'])
    temp = pd.DataFrame(columns=['date', 'open', 'high', 'low', 'close', 'volume'])

    while True:
        if from_date.date() >= (dt.date.today() - dt.timedelta(100)):
            temp=pd.DataFrame(kite.historical_data(instrument,from_date,dt.date.today(),interval))
            data=pd.concat([data,temp])
#             data = data.append(pd.DataFrame(kite.historical_data(instrument,from_date,dt.date.today(),interval)),ignore_index=True)
            #time.sleep(0.2)
            break
        else:
            to_date = from_date + dt.timedelta(100)
            temp=pd.DataFrame(kite.historical_data(instrument,from_date,to_date,interval))
            data=pd.concat([data,temp])
#             data = data.append(pd.DataFrame(kite.historical_data(instrument,from_date,to_date,interval)),ignore_index=True)
            from_date = to_date
            #time.sleep(0.2)
    data.set_index("date",inplace=True)
    return data

In [14]:
def atr(DF,n):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df['H-L']=abs(df['high']-df['low'])
    df['H-PC']=abs(df['high']-df['close'].shift(1))
    df['L-PC']=abs(df['low']-df['close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].ewm(com=n,min_periods=n).mean()
    return df['ATR']

In [15]:

def supertrend(DF,n,m):
    """function to calculate Supertrend given historical candle data
        n = n day ATR - usually 7 day ATR is used
        m = multiplier - usually 2 or 3 is used"""
    df = DF.copy()
    df['ATR'] = atr(df,n)
    df["B-U"]=((df['high']+df['low'])/2) + m*df['ATR'] 
    df["B-L"]=((df['high']+df['low'])/2) - m*df['ATR']
    df["U-B"]=df["B-U"]
    df["L-B"]=df["B-L"]
    ind = df.index
    
    for i in range(n,len(df)):
        if df['close'][i-1]<=df['U-B'][i-1]:
            df.loc[ind[i],'U-B']=min(df['B-U'][i],df['U-B'][i-1])
        else:
            df.loc[ind[i],'U-B']=df['B-U'][i]  
            
    for i in range(n,len(df)):
        if df['close'][i-1]>=df['L-B'][i-1]:
            df.loc[ind[i],'L-B']=max(df['B-L'][i],df['L-B'][i-1])
        else:
            df.loc[ind[i],'L-B']=df['B-L'][i]  
            
    df[ 'Strend'+str(n)]=np.nan
    
    for test in range(n,len(df)):
        if df['close'][test-1]<=df['U-B'][test-1] and df['close'][test]>df['U-B'][test]:
            df.loc[ind[test], 'Strend'+str(n)]=df['L-B'][test]
            break
        if df['close'][test-1]>=df['L-B'][test-1] and df['close'][test]<df['L-B'][test]:
            df.loc[ind[test], 'Strend'+str(n)]=df['U-B'][test]
            break
            
    for i in range(test+1,len(df)):
        if df[ 'Strend'+str(n)][i-1]==df['U-B'][i-1] and df['close'][i]<=df['U-B'][i]:
            df.loc[ind[i], 'Strend'+str(n)]=df['U-B'][i]
        elif  df[ 'Strend'+str(n)][i-1]==df['U-B'][i-1] and df['close'][i]>=df['U-B'][i]:
            df.loc[ind[i], 'Strend'+str(n)]=df['L-B'][i]
        elif df[ 'Strend'+str(n)][i-1]==df['L-B'][i-1] and df['close'][i]>=df['L-B'][i]:
            df.loc[ind[i], 'Strend'+str(n)]=df['L-B'][i]
        elif df[ 'Strend'+str(n)][i-1]==df['L-B'][i-1] and df['close'][i]<=df['L-B'][i]:
            df.loc[ind[i], 'Strend'+str(n)]=df['U-B'][i]
    df.drop(['ATR' , 'B-U' , 'B-L' , 'U-B' , 'L-B'], axis=1, inplace=True)
    return df


In [16]:
def EMA(DF, period):
    df=DF.copy()
    df['ma_'+str(period)]=round(df.iloc[:,0].ewm(span=period, adjust=False).mean(),2)
    return df

In [17]:
def MACD(DF,a,b,c):
    df = DF.copy()
    df["MA_Fast"]=df["close"].ewm(span=a,min_periods=a).mean()
    df["MA_Slow"]=df["close"].ewm(span=b,min_periods=b).mean()
    df["MACD"]=df["MA_Fast"]-df["MA_Slow"]
    df["Signal"]=df["MACD"].ewm(span=c,min_periods=c).mean()
    df.dropna(inplace=True)
    return df

In [18]:
def rsi(df, n):
    "function to calculate RSI"
    delta = df["close"].diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[n-1]] = np.mean( u[:n]) # first value is average of gains
    u = u.drop(u.index[:(n-1)])
    d[d.index[n-1]] = np.mean( d[:n]) # first value is average of losses
    d = d.drop(d.index[:(n-1)])
    rs = u.ewm(com=n,min_periods=n).mean()/d.ewm(com=n,min_periods=n).mean()
    df['rsi_'+str(n)]=100 - 100 / (1+rs)
    return df

In [19]:
def report(log,withdraw,days,cap):
    rep={}
    rep['Stock']=log['ticker'][0]
    rep['Capital']=cap
    rep['Profit']=round(log['profit'].sum(),1)
    rep['Trades']=len(log.index)/2
    rep['Sum of Profit']=round(log['profit'].loc[log['profit']>0].sum(),1)
    rep['Sum of Loss']=round(log['profit'].loc[log['profit']<0].sum(),1)
    rep['Max Profit']=round(log['profit'].max(),1)
    rep['Max Loss']=round(log['profit'].min(),1)
#     return(pd.DataFrame.from_dict(rep))
    return(rep)

In [45]:
def trade(ticker,capital,position,log,comp_flag):
    start_time = time.time()
    df=fetchOHLCExtended(ticker,'27-04-2022',"5minute")
    df2=fetchOHLCExtended(ticker,'27-04-2022',"day")
    df2=EMA(df2,200)
    df2=EMA(df2,26)
    df=supertrend(df,10,3)
    df=supertrend(df,30,9)

    df=EMA(df,200)
    df.to_csv("Signal.csv")
    profit=0
    withdraw={}
    curr_cap=capital
    
    df["time"]=df.index
    df['time'] =pd.to_datetime(df['time'])
    df['dat'] =df['time'].dt.date
    df['time']=df['time'].dt.time
    
    df2["dat"]=df2.index
    df2['dat'] =pd.to_datetime(df2['dat']).dt.date
    
    df2['s200']=df2['close']>=df2['ma_200']
    df2['s26']=df2['close']>=df2['ma_26']

    t1 = pd.Timestamp('2022-03-16 09:30:00').time()
    t2 = pd.Timestamp('2022-03-16 15:15:00').time()
    t3 = pd.Timestamp('2022-04-27')
    
    df=df[(df['time']>=t1)]
    df=df[(df['time']<=t2)]
    
    
#     df = df.merge(df2[['s200','dat','s26']], on='dat')
    
    df=df[df['dat']>=t3]
    
    lap=time.time()
    
    for i in range(1,len(df.index)):
        if(position==0):
            if ((df['close'][i]>df['Strend10'][i-1]) and (df['close'][i]>df['ma_200'][i]) and (df['close'][i]<df['Strend10'][i-1]*1.005) and (df['close'][i]>df['Strend30'][i-1])):
                quantity=(curr_cap/df['close'][i])//1
                bp=quantity*df['close'][i]
                temp=pd.DataFrame({'ticker':ticker,'action': "Buy",'price': df['close'][i],'quantity':quantity,'current capital':curr_cap,'profit':0 },index=[df.index[i]])
                log=pd.concat([log,temp])
                position=1
        else:
            if((df['close'][i]<log['price'][-1]*0.995)):
                sp= quantity*df['close'][i]
                profit=sp-bp
                if comp_flag:
                    curr_cap+=profit
                else:
                    curr_cap+=profit
                    if curr_cap>capital:
                        withdraw[str(df.index[i])]=curr_cap-capital
                        curr_cap=capital
                temp=pd.DataFrame({'ticker':ticker,'action': "Stop",'price': df['close'][i],'quantity':quantity,'current capital':curr_cap,'profit':profit },index=[df.index[i]])
                log=pd.concat([log,temp])
                position=0
                
            elif((df['close'][i]<df['Strend10'][i-1])):
                sp= quantity*df['close'][i]
                profit=sp-bp
                if comp_flag:
                    curr_cap+=profit
                else:
                    curr_cap+=profit
                    if curr_cap>capital:
                        withdraw[str(df.index[i])]=curr_cap-capital
                        curr_cap=capital
                temp=pd.DataFrame({'ticker':ticker,'action': "Sell",'price': df['close'][i],'quantity':quantity,'current capital':curr_cap,'profit':profit },index=[df.index[i]])
                log=pd.concat([log,temp])
                position=0

#             elif((df['time'][i]==t2)):
#                 sp= quantity*df['close'][i]
#                 profit=sp-bp
#                 if comp_flag:
#                     curr_cap+=profit
#                 else:
#                     curr_cap+=profit
#                     if curr_cap>capital:
#                         withdraw[str(df.index[i])]=curr_cap-capital
#                         curr_cap=capital
#                 temp=pd.DataFrame({'ticker':ticker,'action': "Auto",'price': df['close'][i],'quantity':quantity,'current capital':curr_cap,'profit':profit },index=[df.index[i]])
#                 log=pd.concat([log,temp])
#                 position=0
    end_time = time.time() 
    time1=end_time-lap
    time2 = end_time - start_time

    return (log,withdraw,time1,time2)
        

In [46]:
log=pd.DataFrame()
log,withdraw,t1,t2=trade("TATAMOTORS",100000,0,log,True)
print(t1)
print(t2)
log

C:\Users\ronaa\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


0.2884242534637451
10.64253282546997


ticker action   price  quantity  \
2022-04-28 10:30:00+05:30  TATAMOTORS    Buy  431.80     231.0   
2022-04-28 15:10:00+05:30  TATAMOTORS   Sell  435.05     231.0   
2022-04-29 09:30:00+05:30  TATAMOTORS    Buy  440.70     228.0   
2022-04-29 14:20:00+05:30  TATAMOTORS   Sell  443.05     228.0   
2022-05-13 14:00:00+05:30  TATAMOTORS    Buy  407.65     248.0   
...                               ...    ...     ...       ...   
2023-05-10 14:05:00+05:30  TATAMOTORS    Buy  507.40     265.0   
2023-05-11 09:35:00+05:30  TATAMOTORS   Sell  508.80     265.0   
2023-05-11 13:00:00+05:30  TATAMOTORS    Buy  508.95     265.0   
2023-05-12 11:40:00+05:30  TATAMOTORS   Sell  513.50     265.0   
2023-05-12 14:10:00+05:30  TATAMOTORS    Buy  516.85     263.0   

                           current capital   profit  
2022-04-28 10:30:00+05:30        100000.00     0.00  
2022-04-28 15:10:00+05:30        100750.75   750.75  
2022-04-29 09:30:00+05:30        100750.75     0.00  
2022-04-29 14:20:00+05:30        101286.55   535.80  
2022-05-13 14:00:00+05:30        101286.55     0.00  
...                                    ...      ...  
2023-05-10 14:05:00+05:30        134549.70     0.00  
2023-05-11 09:35:00+05:30        134920.70   371.00  
2023-05-11 13:00:00+05:30        134920.70     0.00  
2023-05-12 11:40:00+05:30        136126.45  1205.75  
2023-05-12 14:10:00+05:30        136126.45     0.00  

[265 rows x 6 columns]

In [47]:
print(t1)
print(t2)
report(log,withdraw,1,100000)

0.2884242534637451
10.64253282546997


{'Stock': 'TATAMOTORS',
 'Capital': 100000,
 'Profit': 36126.5,
 'Trades': 132.5,
 'Sum of Profit': 89785.0,
 'Sum of Loss': -53658.5,
 'Max Profit': 10372.2,
 'Max Loss': -4170.0}

In [103]:
log['action'].value_counts()

Buy     125
Sell     86
Stop     38
Name: action, dtype: int64

In [37]:
log.to_csv("Sample_log.csv")

In [14]:

db = sqlite3.connect('C:/Users/ronaa/Documents/Kiteconnect/test.db')

def get_hist(ticker,db):
    token = instrumentLookup(inst_df,ticker)
    data = pd.read_sql('''SELECT * FROM TOKEN%s WHERE ts >=  date() - '12 day';''' %token, con=db)                
    data = data.set_index(['ts'])
    data.index = pd.to_datetime(data.index)
    ticks = data.loc[:, ['price']]   
    df=ticks['price'].resample('5min').ohlc().dropna()
    return df


# df=get_hist("ADANIPORTS",db)

In [22]:
df2=fetchOHLCExtended('TATAMOTORS','10-05-2022',"day")
df=fetchOHLCExtended("TATAMOTORS",'10-04-2022',"60minute")


In [96]:
df['test']=df.index
df2["time"]=df2.index
df2['time'] =pd.to_datetime(df2['time']).dt.date
df["time"]=df.index
df['time'] =pd.to_datetime(df['time']).dt.date


In [25]:
df=EMA(df,200)
df

open    high     low   close   volume  ma_200
date                                                                      
2022-04-11 09:15:00+05:30  457.00  458.90  454.25  454.80  5000272  457.00
2022-04-11 10:15:00+05:30  454.80  456.60  454.60  455.00  1402122  456.98
2022-04-11 11:15:00+05:30  454.95  456.25  454.65  455.40   868944  456.96
2022-04-11 12:15:00+05:30  455.40  455.90  452.75  455.15  1689878  456.94
2022-04-11 13:15:00+05:30  455.30  455.40  451.75  452.80  1710662  456.93
...                           ...     ...     ...     ...      ...     ...
2023-05-12 11:15:00+05:30  515.45  515.55  510.30  513.00  2107506  472.64
2023-05-12 12:15:00+05:30  513.00  515.00  512.25  514.45  1479313  473.04
2023-05-12 13:15:00+05:30  514.45  518.05  512.50  516.85  2736401  473.45
2023-05-12 14:15:00+05:30  516.90  519.25  515.35  517.00  3356950  473.88
2023-05-12 15:15:00+05:30  517.00  517.50  512.00  513.80  1500589  474.31

[1877 rows x 6 columns]

In [89]:
t=pd.Timestamp('2023-05-10')
df[df['time']>=t]

C:\Users\ronaa\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


open    high     low   close   volume  \
date                                                                 
2023-05-10 09:15:00+05:30  508.00  512.85  505.25  507.40  4621597   
2023-05-10 10:15:00+05:30  507.40  507.50  506.05  506.55  1092180   
2023-05-10 11:15:00+05:30  506.45  506.85  504.40  505.50  1472064   
2023-05-10 12:15:00+05:30  505.45  507.00  505.05  506.35   993461   
2023-05-10 13:15:00+05:30  506.35  509.90  504.75  508.85  1516197   
2023-05-10 14:15:00+05:30  508.60  510.00  506.80  509.85  1732728   
2023-05-10 15:15:00+05:30  509.90  510.25  509.15  510.05   957407   
2023-05-11 09:15:00+05:30  512.00  512.00  507.55  509.75  2425995   
2023-05-11 10:15:00+05:30  509.70  510.10  506.00  507.10  1600472   
2023-05-11 11:15:00+05:30  507.05  507.60  505.50  506.30  1190088   
2023-05-11 12:15:00+05:30  506.30  512.00  506.05  511.25  1468999   
2023-05-11 13:15:00+05:30  511.40  514.10  510.50  511.55  2180213   
2023-05-11 14:15:00+05:30  511.40  512.25  509.30  511.95  1639520   
2023-05-11 15:15:00+05:30  511.90  512.30  511.30  511.95   865304   
2023-05-12 09:15:00+05:30  513.90  520.00  512.45  518.85  5191187   
2023-05-12 10:15:00+05:30  518.60  520.50  513.75  515.45  3245927   
2023-05-12 11:15:00+05:30  515.45  515.55  510.30  513.00  2107506   
2023-05-12 12:15:00+05:30  513.00  515.00  512.25  514.45  1479313   
2023-05-12 13:15:00+05:30  514.45  518.05  512.50  516.85  2736401   
2023-05-12 14:15:00+05:30  516.90  519.25  515.35  517.00  3356950   
2023-05-12 15:15:00+05:30  517.00  517.50  512.00  513.80  1500589   

                                               test        time  
date                                                             
2023-05-10 09:15:00+05:30 2023-05-10 09:15:00+05:30  2023-05-10  
2023-05-10 10:15:00+05:30 2023-05-10 10:15:00+05:30  2023-05-10  
2023-05-10 11:15:00+05:30 2023-05-10 11:15:00+05:30  2023-05-10  
2023-05-10 12:15:00+05:30 2023-05-10 12:15:00+05:30  2023-05-10  
2023-05-10 13:15:00+05:30 2023-05-10 13:15:00+05:30  2023-05-10  
2023-05-10 14:15:00+05:30 2023-05-10 14:15:00+05:30  2023-05-10  
2023-05-10 15:15:00+05:30 2023-05-10 15:15:00+05:30  2023-05-10  
2023-05-11 09:15:00+05:30 2023-05-11 09:15:00+05:30  2023-05-11  
2023-05-11 10:15:00+05:30 2023-05-11 10:15:00+05:30  2023-05-11  
2023-05-11 11:15:00+05:30 2023-05-11 11:15:00+05:30  2023-05-11  
2023-05-11 12:15:00+05:30 2023-05-11 12:15:00+05:30  2023-05-11  
2023-05-11 13:15:00+05:30 2023-05-11 13:15:00+05:30  2023-05-11  
2023-05-11 14:15:00+05:30 2023-05-11 14:15:00+05:30  2023-05-11  
2023-05-11 15:15:00+05:30 2023-05-11 15:15:00+05:30  2023-05-11  
2023-05-12 09:15:00+05:30 2023-05-12 09:15:00+05:30  2023-05-12  
2023-05-12 10:15:00+05:30 2023-05-12 10:15:00+05:30  2023-05-12  
2023-05-12 11:15:00+05:30 2023-05-12 11:15:00+05:30  2023-05-12  
2023-05-12 12:15:00+05:30 2023-05-12 12:15:00+05:30  2023-05-12  
2023-05-12 13:15:00+05:30 2023-05-12 13:15:00+05:30  2023-05-12  
2023-05-12 14:15:00+05:30 2023-05-12 14:15:00+05:30  2023-05-12  
2023-05-12 15:15:00+05:30 2023-05-12 15:15:00+05:30  2023-05-12

In [29]:
df['sig']=df['close']<=df['ma_200']

In [30]:
df

open    high     low   close   volume  ma_200  \
date                                                                         
2022-04-11 09:15:00+05:30  457.00  458.90  454.25  454.80  5000272  457.00   
2022-04-11 10:15:00+05:30  454.80  456.60  454.60  455.00  1402122  456.98   
2022-04-11 11:15:00+05:30  454.95  456.25  454.65  455.40   868944  456.96   
2022-04-11 12:15:00+05:30  455.40  455.90  452.75  455.15  1689878  456.94   
2022-04-11 13:15:00+05:30  455.30  455.40  451.75  452.80  1710662  456.93   
...                           ...     ...     ...     ...      ...     ...   
2023-05-12 11:15:00+05:30  515.45  515.55  510.30  513.00  2107506  472.64   
2023-05-12 12:15:00+05:30  513.00  515.00  512.25  514.45  1479313  473.04   
2023-05-12 13:15:00+05:30  514.45  518.05  512.50  516.85  2736401  473.45   
2023-05-12 14:15:00+05:30  516.90  519.25  515.35  517.00  3356950  473.88   
2023-05-12 15:15:00+05:30  517.00  517.50  512.00  513.80  1500589  474.31   

                             sig  
date                              
2022-04-11 09:15:00+05:30   True  
2022-04-11 10:15:00+05:30   True  
2022-04-11 11:15:00+05:30   True  
2022-04-11 12:15:00+05:30   True  
2022-04-11 13:15:00+05:30   True  
...                          ...  
2023-05-12 11:15:00+05:30  False  
2023-05-12 12:15:00+05:30  False  
2023-05-12 13:15:00+05:30  False  
2023-05-12 14:15:00+05:30  False  
2023-05-12 15:15:00+05:30  False  

[1877 rows x 7 columns]

In [83]:
df

open    high     low   close   volume                      test  \
0   508.00  512.85  505.25  507.40  4621597 2023-05-10 09:15:00+05:30   
1   507.40  507.50  506.05  506.55  1092180 2023-05-10 10:15:00+05:30   
2   506.45  506.85  504.40  505.50  1472064 2023-05-10 11:15:00+05:30   
3   505.45  507.00  505.05  506.35   993461 2023-05-10 12:15:00+05:30   
4   506.35  509.90  504.75  508.85  1516197 2023-05-10 13:15:00+05:30   
5   508.60  510.00  506.80  509.85  1732728 2023-05-10 14:15:00+05:30   
6   509.90  510.25  509.15  510.05   957407 2023-05-10 15:15:00+05:30   
7   512.00  512.00  507.55  509.75  2425995 2023-05-11 09:15:00+05:30   
8   509.70  510.10  506.00  507.10  1600472 2023-05-11 10:15:00+05:30   
9   507.05  507.60  505.50  506.30  1190088 2023-05-11 11:15:00+05:30   
10  506.30  512.00  506.05  511.25  1468999 2023-05-11 12:15:00+05:30   
11  511.40  514.10  510.50  511.55  2180213 2023-05-11 13:15:00+05:30   
12  511.40  512.25  509.30  511.95  1639520 2023-05-11 14:15:00+05:30   
13  511.90  512.30  511.30  511.95   865304 2023-05-11 15:15:00+05:30   
14  513.90  520.00  512.45  518.85  5191187 2023-05-12 09:15:00+05:30   
15  518.60  520.50  513.75  515.45  3245927 2023-05-12 10:15:00+05:30   
16  515.45  515.55  510.30  513.00  2107506 2023-05-12 11:15:00+05:30   
17  513.00  515.00  512.25  514.45  1479313 2023-05-12 12:15:00+05:30   
18  514.45  518.05  512.50  516.85  2736401 2023-05-12 13:15:00+05:30   
19  516.90  519.25  515.35  517.00  3356950 2023-05-12 14:15:00+05:30   
20  517.00  517.50  512.00  513.80  1500589 2023-05-12 15:15:00+05:30   

          time  ma_2_x  ma_2_y  
0   2023-05-10  508.00  508.00  
1   2023-05-10  507.60  508.00  
2   2023-05-10  506.83  508.00  
3   2023-05-10  505.91  508.00  
4   2023-05-10  506.20  508.00  
5   2023-05-10  507.80  508.00  
6   2023-05-10  509.20  508.00  
7   2023-05-11  511.07  510.67  
8   2023-05-11  510.16  510.67  
9   2023-05-11  508.09  510.67  
10  2023-05-11  506.90  510.67  
11  2023-05-11  509.90  510.67  
12  2023-05-11  510.90  510.67  
13  2023-05-11  511.57  510.67  
14  2023-05-12  513.12  512.82  
15  2023-05-12  516.77  512.82  
16  2023-05-12  515.89  512.82  
17  2023-05-12  513.96  512.82  
18  2023-05-12  514.29  512.82  
19  2023-05-12  516.03  512.82  
20  2023-05-12  516.68  512.82

In [58]:
merged_df

open    high     low   close   volume        time  ma_200  mad_200
0      295.70  297.15  295.15  296.50  1719456  2021-04-27  295.70   295.70
1      296.30  296.75  295.80  296.45   869742  2021-04-27  295.71   295.70
2      296.40  296.45  295.50  295.65   558522  2021-04-27  295.71   295.70
3      295.65  296.40  295.50  296.05   524640  2021-04-27  295.71   295.70
4      296.00  296.60  295.90  296.40   504728  2021-04-27  295.72   295.70
...       ...     ...     ...     ...      ...         ...     ...      ...
38343  517.95  518.65  516.80  517.00   333234  2023-05-12  510.56   436.11
38344  517.05  517.50  516.70  517.00   276959  2023-05-12  510.62   436.11
38345  517.00  517.50  516.50  516.55   423280  2023-05-12  510.69   436.11
38346  516.45  516.60  513.15  513.60   517300  2023-05-12  510.74   436.11
38347  513.55  513.90  512.00  513.80   560009  2023-05-12  510.77   436.11

[38348 rows x 8 columns]

In [ ]:
# plt.figure().set_figwidth(20)
# plt.figure().set_figheight(5)
plt.figure(figsize=(20,6))
plt.plot(df['close'], label = "close")
plt.plot(df['ma_200'], label = "open")
plt.plot(df['Strend'], label = "open")
# plt.plot(df.index, df['close'], label = "close")
# plt.plot(df.index, df['ma_200'], label = "open")
# plt.plot(df.index, df['Strend'], label = "open")

In [91]:
log['profit'].values.argmin()


121

In [95]:
log.iloc[121]

ticker             TATAMOTORS
action                   Sell
price                  435.05
quantity               1621.0
current capital     705268.25
profit                5268.25
Name: 2022-04-28 15:10:00+05:30, dtype: object

In [93]:
log.iloc[120]

ticker             TATAMOTORS
action                    Buy
price                   411.2
quantity               1995.0
current capital     820570.45
profit                    0.0
Name: 2022-10-07 10:35:00+05:30, dtype: object